# MSDS 7337 NLP Homework 6

## Evangelos Giakoumakis

In [1]:
# get system information
import platform; print platform.platform()
import sys; print "Python", sys.version
import nltk; print "nltk", nltk.__version__
import requests; print "requests", requests.__version__
import re; print "re", re.__version__
import os; print os.environ['CONDA_DEFAULT_ENV']


Windows-10-10.0.17134
Python 2.7.15 |Anaconda, Inc.| (default, May  1 2018, 18:37:09) [MSC v.1500 64 bit (AMD64)]
nltk 3.3
requests 2.18.4
re 2.2.1
base


In [2]:
# imports
from bs4 import BeautifulSoup
import requests
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [3]:
# amazon book links
am_url = "https://www.amazon.com/s/ref=nb_sb_ss_c_1_5?url=search-alias%3Dstripbooks&field-keywords=game+of+thrones&sprefix=game+%2Cundefined%2C186&crid=6O1IHGE860SY&rh=n%3A283155%2Ck%3Agame+of+thrones"
am_url2 = "https://www.amazon.com/s/ref=sr_pg_2?rh=n%3A283155%2Ck%3Agame+of+thrones&page=2&keywords=game+of+thrones&ie=UTF8&qid=1541373342"


In [4]:
# amazon scraper function
def amazon_scrapper(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.content, "lxml")
    titles = soup.find_all('div', class_ = 'a-row a-spacing-small')
    results = []
    for tl in titles:
        results.append(tl.text)
    return results

In [6]:
# collect book titles of first 2 pages and place in list
got_pg1 = []
got_pg1 = amazon_scrapper(am_url)
got_pg1

[u'A Game of Thrones (A Song of Ice and Fire, Book 1)Mar 22, 2011by George R. R. Martin',
 u'A Clash of Kings (A Song of Ice and Fire, Book 2)Feb 2, 1999by George R. R. Martin',
 u'A Storm of Swords (A Song of Ice and Fire, Book 3)Mar 4, 2003by George R. R. Martin',
 u'A Feast for Crows (A Song of Ice and Fire, Book 4)Nov 8, 2005by George R. R. Martin',
 u'A Dance with Dragons (A Song of Ice and Fire, Book 5)Jul 12, 2011by George R. R. Martin',
 u'A Game of Thrones / A Clash of Kings / A Storm of Swords / A Feast of Crows / A Dance with Dragons Oct 29, 2013by George R. R. Martin',
 u'Fire & Blood: 300 Years Before A Game of Thrones (A Targaryen History) (A Song of Ice and Fire)Nov 20, 2018by George R. R. Martin and Doug Wheatley',
 u'Game of Thrones: A Pop-Up Guide to WesterosJun 10, 2014by Matthew Reinhart and Michael J. Komarck',
 u'The World of Ice & Fire: The Untold History of Westeros and the Game of Thrones Oct 28, 2014by George R. R. Martin and Elio Garcia',
 u'A Game of Thrones

In [7]:
got_pg2 = []
got_pg2 = amazon_scrapper(am_url2)
got_pg2

[u'[Sponsored]The Ukinhan Wilds (Warder Book 1)Aug 28, 2018by Eldon Thompson',
 u'To return a wayward princess to her royal father, a rogue bladesman must battle a savage mutant and the harrowing wilderness that spawned it.',
 u'[Sponsored]The Dark Side of Light:  Book One-INITIATION: A Viking Time Travel Fantasy (The Dark Side of Light Trilogy 1)May 16, 2018by Susan D. Kalior',
 u'When two souls are fated to be together, not even time and space can keep them apart.',
 u'A Feast for Crows (A Song of Ice and Fire, Book 4)Nov 8, 2005by George R. R. Martin',
 u'Game of Thrones TarotMar 20, 2018by Liz Dean and Craig Coss',
 u'Brain Of Thrones: A Game Of Thrones Quiz BookSep 6, 2017by Rich Jepson',
 u'The Official A Game of Thrones Coloring Book: An Adult Coloring Book (A Song of Ice and Fire)Oct 27, 2015by George R. R. Martin',
 u'A Song of Ice & Fire RPG: A Game of Thrones EditionJun 12, 2012by Robert J. Schwalb',
 u'The Unofficial Game of Thrones Cookbook: From Direwolf Ale to Auroch Ste

In [11]:
# combine collected results and eliminate sponsored books
got = []
got = got_pg1[:11] + got_pg2[4:17]
print len(got)

24


In [12]:
print "1. a) 24 Amazon Book titles: "
print got

1. a) 24 Amazon Book titles: 
[u'A Game of Thrones (A Song of Ice and Fire, Book 1)Mar 22, 2011by George R. R. Martin', u'A Clash of Kings (A Song of Ice and Fire, Book 2)Feb 2, 1999by George R. R. Martin', u'A Storm of Swords (A Song of Ice and Fire, Book 3)Mar 4, 2003by George R. R. Martin', u'A Feast for Crows (A Song of Ice and Fire, Book 4)Nov 8, 2005by George R. R. Martin', u'A Dance with Dragons (A Song of Ice and Fire, Book 5)Jul 12, 2011by George R. R. Martin', u'A Game of Thrones / A Clash of Kings / A Storm of Swords / A Feast of Crows / A Dance with Dragons Oct 29, 2013by George R. R. Martin', u'Fire & Blood: 300 Years Before A Game of Thrones (A Targaryen History) (A Song of Ice and Fire)Nov 20, 2018by George R. R. Martin and Doug Wheatley', u'Game of Thrones: A Pop-Up Guide to WesterosJun 10, 2014by Matthew Reinhart and Michael J. Komarck', u'The World of Ice & Fire: The Untold History of Westeros and the Game of Thrones Oct 28, 2014by George R. R. Martin and Elio Garcia'

In [17]:
# pairwise cosine similarity function
def cos_sim(intxt):
    tfidf_vectorizer=TfidfVectorizer()
    tfidf_matrix=tfidf_vectorizer.fit_transform(intxt)
    cs = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    for i in range(len(intxt)-1):
        cs[i]=cosine_similarity(tfidf_matrix[i],tfidf_matrix[i+1])
    return cs

In [20]:
# calculate cosine similarity and place results on list
got_cos_sim = cos_sim(got)

In [22]:
print "1. b) Cosine Similarity measurements: "
print got_cos_sim

1. b) Cosine Similarity measurements: 
[array([[0.36387458]]), array([[0.34683375]]), array([[0.3261268]]), array([[0.27823025]]), array([[0.35304913]]), array([[0.12056796]]), array([[0.06615533]]), array([[0.1402368]]), array([[0.27854425]]), array([[0.12881571]]), array([[0.02664951]]), array([[0.04065381]]), array([[0.09446055]]), array([[0.09466687]]), array([[0.156706]]), array([[0.07931479]]), array([[0.02389383]]), array([[0.08099637]]), array([[0.0368865]]), array([[0.01953819]]), array([[0.]]), array([[0.26144552]]), array([[0.15970311]])]


In [24]:
print "1. c) Book titles 1-2 are the most similar (cos_sim: 0,3639) followed by books 2-3 (cos_sim: 0.3468)"
print "Books 20-21 are the most disimilar (cos_sim: 0.0) followed by books 19-20 (cos_sim: 0.0195)"

1. c) Book titles 1-2 are the most similar (cos_sim: 0,3639) followed by books 2-3 (cos_sim: 0.3468)
Books 20-21 are the most disimilar (cos_sim: 0.0) followed by books 19-20 (cos_sim: 0.0195)


In [25]:
# google book links
original = "Game of Thrones"
gl_url = "https://www.google.com/search?q=game+of+thrones&source=lnms&tbm=bks&sa=X&ved=0ahUKEwiytczTqbneAhXumOAKHeOUC-EQ_AUIGSgB&biw=1242&bih=569&dpr=1.1"
gl_url2 = "https://www.google.com/search?q=game+of+thrones&tbm=bks&ei=RaPfW_KXKtHIsAXtjbWoCw&start=10&sa=N&ved=0ahUKEwiyztiIk7zeAhVRJKwKHe1GDbUQ8tMDCGg&biw=737&bih=569&dpr=1.1"


In [26]:
# google scraper function
def google_scrapper(url):
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/54.0.2840.71 Safari/537.36'}
    r = requests.get(url, headers=headers)
    soup = BeautifulSoup(r.content, "lxml")
    titles = soup.find_all('div', class_ = 'rc')
    results = []
    for tl in titles:
        results.append(tl.text)
    return results

In [27]:
# collect book titles of first 2 pages and place in list
glg_got = []
glg_got = google_scrapper(gl_url)
glg_got

[u"A Game of Thrones 4-Book Bundle: A Song of Ice and Fire Series: A ...https://books.google.com/books?isbn=0345529065George R. R. Martin - 2011 - \u200ePreviewThe perfect gift for fans of HBO's Game of Thrones\u2014a boxed set featuring the first four novels! George R. R. Martin's A Song of Ice and Fire series has become, in many ways, the gold standard for modern epic fantasy.",
 u'A Game of Throneshttps://books.google.com/books?isbn=0553808044George R. R. Martin - 2016 - \u200ePreview - \u200eMore editionsSweeping from a region of brutal cold to a distant summertime kingdom of epicurean plenty, here is a tale rife with plots and counterplots, tragedy and betrayal, victory and terror, as each faction endeavors to win that deadliest of ...',
 u'Game of Thrones: A Pop-Up Guide to Westeroshttps://books.google.com/books?isbn=1608873145Matthew Reinhart - 2014 - \u200eNo preview - \u200eMore editionsVisually spectacular and enthrallingly interactive, Game of Thrones: A Pop-Up Guide to West

In [28]:
glg_got2 = []
glg_got2 = google_scrapper(gl_url2)
glg_got2

[u"Inside HBO's Game of Thrones: Seasons 3 & 4https://books.google.com/books?isbn=1452130477C. A. Taylor - 2014 - \u200ePreview - \u200eMore editionsThis official companion book reveals what it takes to translate George R. R. Martin's bestselling series into a wildly popular television series.",
 u'Game of Thrones: In Memoriamhttps://books.google.com/books?isbn=0762457910Running Press - 2015 - \u200ePreview - \u200eMore editionsThe book includes quotes, brief character profiles, and is fully illustrated with series photography throughout.',
 u"Winning the Game of Thrones: The Host of Characters and Their Agendashttps://books.google.com/books?isbn=0615817440Valerie Estelle Frankel - 2013 - \u200eNo preview - \u200eMore editionsThis book explores characters' agendas, from Sansa's secret schemes to Arya's ultimate goal.",
 u'A Game of Thrones: The Illustrated Edition: A Song of Ice and Fire: ...https://books.google.com/books?isbn=1101965878George R. R. Martin - 2016 - \u200ePreview - \u20

In [29]:
# combine results and place in list
glg_fin = []
glg_fin = glg_got + glg_got2
len(glg_fin)

20

In [30]:
print "2. a) Google Book titles: "
print glg_fin[0]
print glg_fin[19]

2. a) Google Book titles: 
A Game of Thrones 4-Book Bundle: A Song of Ice and Fire Series: A ...https://books.google.com/books?isbn=0345529065George R. R. Martin - 2011 - ‎PreviewThe perfect gift for fans of HBO's Game of Thrones—a boxed set featuring the first four novels! George R. R. Martin's A Song of Ice and Fire series has become, in many ways, the gold standard for modern epic fantasy.
The Winds of Winterhttps://books.google.com/books?isbn=0008165165George R. R. Martin - No preview - ‎More editionsThe sixth book in George R. R. Martin's critically acclaimed, world wide best-selling series A SONG OF ICE AND FIRE - the inspiration behind HBO's GAME OF THRONES.


In [31]:
# add original search query to list
glg_fin = glg_fin + [original]
glg_fin

[u"A Game of Thrones 4-Book Bundle: A Song of Ice and Fire Series: A ...https://books.google.com/books?isbn=0345529065George R. R. Martin - 2011 - \u200ePreviewThe perfect gift for fans of HBO's Game of Thrones\u2014a boxed set featuring the first four novels! George R. R. Martin's A Song of Ice and Fire series has become, in many ways, the gold standard for modern epic fantasy.",
 u'A Game of Throneshttps://books.google.com/books?isbn=0553808044George R. R. Martin - 2016 - \u200ePreview - \u200eMore editionsSweeping from a region of brutal cold to a distant summertime kingdom of epicurean plenty, here is a tale rife with plots and counterplots, tragedy and betrayal, victory and terror, as each faction endeavors to win that deadliest of ...',
 u'Game of Thrones: A Pop-Up Guide to Westeroshttps://books.google.com/books?isbn=1608873145Matthew Reinhart - 2014 - \u200eNo preview - \u200eMore editionsVisually spectacular and enthrallingly interactive, Game of Thrones: A Pop-Up Guide to West

In [32]:
# first-last comparison with original query cosine similarity function
def glg_cos_sim(intxt):
    tfidf_vectorizer=TfidfVectorizer()
    tfidf_matrix=tfidf_vectorizer.fit_transform(intxt)
    cs1=cosine_similarity(tfidf_matrix[20],tfidf_matrix[0])
    cs2=cosine_similarity(tfidf_matrix[20],tfidf_matrix[19])
    return cs1, cs2

In [33]:
# calculate cosine similarity and place results on variable
glg_cos_sin = glg_cos_sim(glg_fin)

In [34]:
print "2. b) Text Similarity for Google search: "
print "Google Cosine Similarity: ", glg_cos_sin
print "Amazon Cosine Similarity: ", got_cos_sim

2. b) Text Similarity for Google search: 
Google Cosine Similarity:  (array([[0.2719392]]), array([[0.20712048]]))
Amazon Cosine Similarity:  [array([[0.36387458]]), array([[0.34683375]]), array([[0.3261268]]), array([[0.27823025]]), array([[0.35304913]]), array([[0.12056796]]), array([[0.06615533]]), array([[0.1402368]]), array([[0.27854425]]), array([[0.12881571]]), array([[0.02664951]]), array([[0.04065381]]), array([[0.09446055]]), array([[0.09466687]]), array([[0.156706]]), array([[0.07931479]]), array([[0.02389383]]), array([[0.08099637]]), array([[0.0368865]]), array([[0.01953819]]), array([[0.]]), array([[0.26144552]]), array([[0.15970311]])]


In [35]:
print "2. c) Highest Similarity measure found on first results of amazon book search."
print  "As we can see from the results above the google cosine similarity is higher for the first book than the last one. However our amazon books results held the highest cosine similarity meausure. It seems that results on the beginning of each page hold the highest similarity as is expected from a good ranking algorithm. Additionally one can see that amazon returned more accurate results as opposed to google"


2. c) Highest Similarity measure found on first results of amazon book search.
As we can see from the results above the google cosine similarity is higher for the first book than the last one. However our amazon books results held the highest cosine similarity meausure. It seems that results on the beginning of each page hold the highest similarity as is expected from a good ranking algorithm. Additionally one can see that amazon returned more accurate results as opposed to google
